In [1]:
import numpy as np
import pandas as pd
import random
import re
import matplotlib.pyplot as plt

%matplotlib qt


In [2]:
df = pd.read_csv("../Data/Cleaned/from_kickstarter_reduced.csv")

In [3]:
df=df[["blurb","binary_state"]]
df=df[:8000]
df.head()

,blurb,binary_state
0,"Local, Delicious food themed from around the g...",0
1,Drawing life is aiming to set up a workspace f...,0
2,"Hello everyone, I'm Anastasia Humphrey. If you...",0
3,A gruesome underground graphic novel that slic...,0
4,"An all-day festival with exhibits, competition...",0


In [4]:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df['blurb'] = df['blurb'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df['blurb'] = df['blurb'].apply(lambda x: lower_case(x))
df.head()

,blurb,binary_state
0,local delicious food themed from around the gl...,0
1,drawing life is aiming to set up a workspace f...,0
2,hello everyone im anastasia humphrey if you de...,0
3,a gruesome underground graphic novel that slic...,0
4,an allday festival with exhibits competitions ...,0


In [5]:
text=df.drop(['binary_state'],axis=1)
text_arr = text.stack().tolist()


In [6]:
words = []

for ii in range(0,len(text)):
    words.append(str(text.iloc[ii]['blurb']).split(" "))


In [7]:
ctext=[]
for s in words:
    ctext+=s
ctext_df=pd.DataFrame(ctext)
ctext_df.shape,ctext_df.nunique()

((152247, 1),
 0    20542
 dtype: int64)

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
ctext = np.array(ctext)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(ctext)
map=pd.DataFrame([ctext,integer_encoded])

    
#text.head()
#print(map)
# binary encode

In [9]:
map.shape

(2, 152247)

In [10]:
new_header = map.iloc[0] #grab the first row for the header
map = map[1:] #take the data less the header row
map.columns = new_header #set the header row as the df header
map.head()

,local,delicious,food,themed,from,around,the,globe,,family,...,,studio,space,for,artists,,wholesale,and,retail,ceramics
1,10941,5149,7430,18318,7651,1599,18302,8068,0,6943,...,0,17653,17177,7453,1652,0,20050,1302,15437,3461


In [11]:
map=map.to_dict('r')[0]
a=pd.DataFrame(['local','local'])
print(a)
a.replace(map,inplace=True)
print(a)

<ipython-input-11-4a84cb24cebc>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  map=map.to_dict('r')[0]


       0
0  local
1  local
       0
0  10941
1  10941


In [12]:
def replace(list,map):
    for i,w in enumerate(list):
        if w in map:
            list[i]=map[w]
        else:
            list[i]=0
    vector={}
    for f in list:
        if not f in vector:
            vector[f]=1
        else:
            vector[f]+=1
    return vector


In [13]:
vectors=[]
for s in words:
    vectors+=[replace(s,map)]
vectors[:1]


[{10941: 1,
  5149: 1,
  7430: 1,
  18318: 1,
  7651: 1,
  1599: 1,
  18302: 2,
  8068: 1,
  0: 2,
  6943: 1,
  17669: 1,
  5461: 1,
  6768: 1,
  18368: 1,
  9839: 1,
  17363: 1,
  13017: 1,
  12929: 1,
  7721: 1,
  14441: 1}]

In [14]:
def onehotvectorize(v,vlength):
    hotvector=[]
    for i in range(vlength):
        if i in v:
            hotvector+=[v[i]]
        else:
            hotvector+=[0]
    a= np.array(hotvector)
    a=a/np.linalg.norm(a)
    return a.tolist()
onehotvectorize({1:2,2:3,4:1},8)
            
            

[0.0,
 0.5345224838248488,
 0.8017837257372732,
 0.0,
 0.2672612419124244,
 0.0,
 0.0,
 0.0]

In [15]:
blurbvectors=[]
vectorlength=ctext_df.nunique()[0]
print(vectorlength)


20542


In [16]:
for v in vectors:
    blurbvectors+=[onehotvectorize(v,vectorlength)]

In [17]:
blurbvectors=np.array(blurbvectors)
blurbvectors.shape

(8000, 20542)

In [18]:
from sklearn.decomposition import PCA
import seaborn as sns

pca = PCA(n_components=200)
pca_result = pca.fit_transform(blurbvectors)

In [19]:
import matplotlib.pyplot as plt
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.show()

In [20]:

# colors
# sns settings

pca_fail=pca_result[df['binary_state']==0,:]
pca_succ=pca_result[df['binary_state']==1,:]

sns.set(rc={'figure.figsize':(15,15)})
fail = sns.color_palette("bright", 1)
succ= sns.color_palette("dark", 1)
# plot
#sns.scatterplot(X_embedded[:,0], X_embedded[:,1], palette=fail)
sns.scatterplot(pca_fail[:,0], pca_fail[:,1], palette=fail)
sns.scatterplot(pca_succ[:,0], pca_succ[:,1], palette=succ)

plt.title("Kickstarter description")
# plt.savefig("plots/t-sne_covid19.png")
plt.show()

In [21]:
#%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

ax = plt.figure(figsize=(16,10)).gca(projection='3d')

ax.scatter(
    xs=pca_succ[:,0], 
    ys=pca_succ[:,1], 
    zs=pca_succ[:,2], 
    c='blue',
    cmap='tab10'
)
ax.scatter(
    xs=pca_fail[:,0], 
    ys=pca_fail[:,1], 
    zs=pca_fail[:,2], 
    c='yellow',
    cmap='tab10'
)


ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.title("PCA Covid-19 Articles (3D) - Clustered (K-Means) - Tf-idf with Plain Text")
# plt.savefig("plots/pca_covid19_label_TFID_3d.png")

ax.view_init(0, 40)
plt.draw()


In [22]:
ax = plt.figure(figsize=(16,10)).gca(projection='3d')

ax.scatter(
    xs=pca_fail[:,0], 
    ys=pca_fail[:,1], 
    zs=pca_fail[:,2], 
    c=fail,
    cmap='tab10'
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.title("PCA Covid-19 Articles (3D) - Clustered (K-Means) - Tf-idf with Plain Text")
# plt.savefig("plots/pca_covid19_label_TFID_3d.png")
plt.show()

In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = np.array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)
print(label_encoder.transform(['cold']))

['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']
[0 0 2 0 1 1 2 0 2 1]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
['cold']
[0]
